# Fake News URL Datasets

There exist lists of fake news domains from three different data sources: **Politifacts**, **OpenSourceGroup**, **Wikipedia**, and **BuzzFeed**. 

In order to reliably identify fake news in our social media research studies, we want to reproducibly generate a data set of fake news domains from each of these data sources.

Therefore we download and process the raw data from each data source in this python notebook, in order to generate unique lists of fake news domains for different research questions.

The process is as follows:

1. Download the different fake news URL datasets

- Clean up data & remove duplicates

- Compare different datasets

- Output as .txt files


## 1) Download fake news URL datasets

The datasets have already been downloaded to this folder. The data sources are documented in the readme text files.


## 2) Clean up data & remove duplicates

In [1]:
import pandas as pd

data_sources = {
    "OpenSourceGroup": ["opensourcesgroup-raw-16Apr2020.csv"],
    "Politifacts": ["politifacts-raw-16Apr2020.csv"],
    "Wikipedia": ["wikipedia-fake-news-raw-16Apr2020.csv", "wikipedia-fake-news-usa-raw-16Apr2020.csv"],
    "BuzzFeed": ["buzzfeed-fakenews-sites_2016.csv", "buzzfeed-fakenews-sites_2017.csv", "buzzfeed-fakenews-sites_2018.csv"]
}

dfs = {}

# function to clean up our domains
def clean_up_entry(str_domain_or_link):
    # if domain is google.com/foo then we only want google.com
    for sep in ['/', '\\', '#', '?']:
        if sep in str_domain_or_link:
            str_domain_or_link = str_domain_or_link.split(sep)[0]
            
    # make everything lowercase
    domain = str_domain_or_link.lower()
                
    # replace www in front of domains
    domain = domain.replace("www.", "")
    
    return domain

# clean up a category (e.g. "fake news", "fake ", " fake" => "fake")
def clean_up_category(str_category):
    # lower-case and remove trailing whitespace
    str_category = str_category.lower().strip()
    
    # only use first word
    first_part = str_category.split(" ")[0]
    
    return first_part

# remove categories that dont make any sense for our "fake news"-focused study
bogus_categories = ["reliable", "political", "some", "state", "clickbait"]
def remove_bogus_categories(set_of_tuples):
    print("  remove_bogus_categories", len(set_of_tuples))
    retval = list(filter(lambda x: bool(x[1] not in bogus_categories), list(set_of_tuples)))
    print("   -> still remaining:", len(retval))
    return retval

# removes duplicate tuples, keeping the most important categories
category_importance = ["fake", "hate", "satire"]
def remove_duplicate_entries(list_of_tuples):
    unique_domains = set([x[0] for x in list_of_tuples])
    ret = []
    for domain in unique_domains:
        unique_categories = list(set([x[1] for x in list(filter(lambda x: x[0] == domain, list_of_tuples))]))
        if len(unique_categories) > 1:
            print("len > 1:", domain, unique_categories)
            selected_category = None
            for category in unique_categories:
                if not selected_category:
                    selected_category = category
                else:
                    if category in category_importance and selected_category not in category_importance:
                        selected_category = category
                    if category not in category_importance and selected_category in category_importance:
                        continue # do nothing
                    if category not in category_importance and selected_category not in category_importance:
                        continue # do nothing
                    if category_importance.index(category) < category_importance.index(selected_category):
                        selected_category = category
            print ("selected_category:", selected_category)
                
        else:
            selected_category = unique_categories[0]
        
        ret.append((domain, selected_category))                  
    return ret


# read all downloaded files into our data structure
for source in data_sources.keys():
    print("\nFrom", source + ":")
    for csv in data_sources[source]:
        # open + read csv file into pandas dataframe
        df = pd.read_csv("data/" + source + "/" + csv)
        
        # domains are always in first column of csv
        first_column = list(df[df.columns[0]])
        
        # clean up domains from rubbish data
        domains = list(map(clean_up_entry, first_column))
        
        # categories are always in second column of csv,         
        # ... except for wikipedia/buzzfeed, here everything should be seen as "fake"
        if source == "Wikipedia" or source == "BuzzFeed":
            categories = ["fake"] * len(domains)
        else:
            # clean up categories
            categories = list(df[df.columns[1]])
            categories = list(map(clean_up_category, categories))
        
        print("  -", len(domains), "entries in", csv)
        
        # zip categories and domains into tuples
        new_data = list(zip(domains, categories))
        
        # store the data (merge with other data for same source if needed)
        if source in dfs.keys():
            dfs[source].extend(new_data)
        else:
            dfs[source] = new_data
    
    # remove duplicate tuples
    dfs[source] = set(sorted(dfs[source]))
    
    # remove bogus categories
    dfs[source] = remove_bogus_categories(dfs[source])
    
    # remove duplicate entries
    dfs[source] = remove_duplicate_entries(dfs[source])
    
    print("  => unique fake news domains:\t", len(dfs[source]))


From OpenSourceGroup:
  - 833 entries in opensourcesgroup-raw-16Apr2020.csv
  remove_bogus_categories 827
   -> still remaining: 723
len > 1: centerforsecuritypolicy.org ['bias', 'hate']
selected_category: hate
len > 1: patriotnewsdaily.com ['bias', 'satire']
selected_category: satire
len > 1: madworldnews.com ['unreliable', 'fake']
selected_category: fake
  => unique fake news domains:	 720

From Politifacts:
  - 327 entries in politifacts-raw-16Apr2020.csv
  remove_bogus_categories 327
   -> still remaining: 277
len > 1: civictribune.com ['imposter', 'fake']
selected_category: fake
  => unique fake news domains:	 276

From Wikipedia:
  - 82 entries in wikipedia-fake-news-raw-16Apr2020.csv
  - 18 entries in wikipedia-fake-news-usa-raw-16Apr2020.csv
  remove_bogus_categories 88
   -> still remaining: 88
  => unique fake news domains:	 88

From BuzzFeed:
  - 96 entries in buzzfeed-fakenews-sites_2016.csv
  - 167 entries in buzzfeed-fakenews-sites_2017.csv
  - 129 entries in buzzfeed-fa

# 3) Compare different datasets

Now that we have parsed the data, cleaned it up and removed the duplicates, we want to compare the datasets.

We want to find out **common fake news domains accross all datasets**.

In [2]:
import numpy as np
from functools import reduce

# calculate list of common domains across all datasets
common_domains_across_all_datasets = reduce(set.intersection, [set([y[0] for y in x]) for x in dfs.values()])

print("Common domains across all data sets:",  len(common_domains_across_all_datasets))
print()
for domain in common_domains_across_all_datasets:
    print("  -", domain)

Common domains across all data sets: 15

  - now8news.com
  - thenewyorkevening.com
  - nationalreport.net
  - empireherald.com
  - abcnews.com.co
  - empirenews.net
  - newsexaminer.net
  - empiresports.co
  - undergroundnewsreport.com
  - christiantimesnewspaper.com
  - thelastlineofdefense.org
  - kmt11.com
  - huzlers.com
  - worldnewsdailyreport.com
  - react365.com


In [3]:
# calculate intersection of domains labeled "fake"

# filter for "fake" category
common_domains_across_datasets_fake_only = [list(filter(lambda x: x[1] == "fake", arr)) for arr in dfs.values()]

# only use domain name
common_domains_across_datasets_fake_only = [set(map(lambda x: x[0], arr)) for arr in common_domains_across_datasets_fake_only]

# perform set intersection
common_domains_across_datasets_fake_only = list(reduce(set.intersection, common_domains_across_datasets_fake_only))

print("Common domains across all data sets (fake only):",  len(common_domains_across_datasets_fake_only))
print()
for domain in common_domains_across_datasets_fake_only:
    print("  -", domain)

Common domains across all data sets (fake only): 2

  - undergroundnewsreport.com
  - newsexaminer.net


In [4]:
# calculate list of all fake news domains
all_domains_fake_only = [set(filter(lambda x: x[1] == "fake", arr)) for arr in dfs.values()]

# perform set union operation
all_domains_fake_only = set(reduce(set.union, [set([x[0] for x in arr]) for arr in all_domains_fake_only]))

print("Total number of 'fake' domains (all):", len(all_domains_fake_only))
print()
for domain in all_domains_fake_only:
    print("  -", domain)

Total number of 'fake' domains (all): 582

  - usasnich.com
  - liberalsociety.com
  - nbcnews.com.co
  - president45donaldtrump.com
  - myzonetoday.com
  - dcgazette.com
  - nationalreport.net
  - квоі2.com
  - usadailytime.com
  - wm21news.com
  - thelastlineofdefense.org
  - newsbreakhere.com
  - usanewshome.com
  - ladylibertysnews.com
  - newsfrompolitics.com
  - countyweekly.com
  - politicalupdator.com
  - newsformetoday.com
  - battypost.com
  - rearfront.com
  - undergroundnewsreport.com
  - dailynews33.com
  - intrendtoday.com
  - thatviralfeed.net
  - huzlers.com
  - federalisttribune.com
  - usfanzone.com
  - thebostontribune.com
  - oreillypost.com
  - breakingnewsblast.com
  - newsmagazine.com
  - redflagnews.com
  - conservativeinfocorner.com
  - veteransfordonaldtrump.com
  - 24x365live.com
  - worldrumor.com
  - en-vovle.com
  - dailyworldupdate.us
  - globalrevolutionnetwork.com
  - usanewstoday.com
  - truetrumpers.com
  - fakingnews.com
  - rumorjournal.com
  - heal

In [5]:
# calculate list of all fake news domains
all_domains = reduce(set.union, 
                               [set([y[0] for y in x]) for x in dfs.values()])

print("Total number of domains (all):", len(all_domains))
print()
for domain in all_domains:
    print("  -", domain)

Total number of domains (all): 1053

  - aheadoftheherd.com
  - awm.com
  - educate-yourself.org
  - politicsintheusa.com
  - usasnich.com
  - thedailysheeple.com
  - liberalsociety.com
  - dcgazette.com
  - nationalreport.net
  - myzonetoday.com
  - president45donaldtrump.com
  - johnnyrobish.com
  - usadailytime.com
  - freedomjunkshun.com
  - nbcnews.com.co
  - theeconomiccollapseblog.com
  - квоі2.com
  - wm21news.com
  - sovereignman.com
  - itccs.org
  - thelastlineofdefense.org
  - pamelageller.com
  - newsbreakhere.com
  - speld.nl
  - usanewshome.com
  - ladylibertysnews.com
  - sentinelblog.com
  - newsfrompolitics.com
  - newsmutiny.com
  - countyweekly.com
  - truthandaction.org
  - politicalupdator.com
  - newsformetoday.com
  - theunrealtimes.com
  - reagancoalition.com
  - rearfront.com
  - battypost.com
  - undergroundnewsreport.com
  - dailynews33.com
  - intrendtoday.com
  - huzlers.com
  - thatviralfeed.net
  - checkoutthehealthyworld.com
  - intellihub.com
  - newst

# 4) Output list of fake news domains as .txt file

In [6]:
FAKE_SUFFIX = "-fake-category-only"

# calculate fake news only categories
new_keys = {}
for key in dfs:
    data = dfs[key]
    
    fake_only = list(filter(lambda x: x[1] == "fake", data))
    new_keys[key + FAKE_SUFFIX] = fake_only
dfs.update(new_keys)


# store data for saving
dfs["all"] = all_domains
dfs["all" + FAKE_SUFFIX] = all_domains_fake_only
dfs["intersection"] = common_domains_across_all_datasets
dfs["intersection" + FAKE_SUFFIX] = common_domains_across_datasets_fake_only

# store all results in csv files
print ("Saving files:\n")

import operator
import os 

outputdir = "output/"
for key in dfs:
    # create filename
    filename = "fake-news-domains-" + key + "-Jul2022.csv"
    outputpath = os.path.join(outputdir, filename)

    # sort the data
    tuples = list(dfs[key])
    tuples.sort(key = operator.itemgetter(0))

    if type(tuples[0]) is tuple:
        domains = [x[0] for x in tuples]
        categories = [x[1] for x in tuples]
    else:
        domains = tuples
        categories = [""] * len(domains)

    # create dataframe
    df = pd.DataFrame({"fake_news_domain": domains, "category": categories })

    # store in csv
    df.to_csv(outputpath, header=True, index=False)

    print(" -", filename, " (" + str(len(dfs[key])) + " entries)")

Saving files:

 - fake-news-domains-OpenSourceGroup-Jul2022.csv  (720 entries)
 - fake-news-domains-Politifacts-Jul2022.csv  (276 entries)
 - fake-news-domains-Wikipedia-Jul2022.csv  (88 entries)
 - fake-news-domains-BuzzFeed-Jul2022.csv  (257 entries)
 - fake-news-domains-OpenSourceGroup-fake-category-only-Jul2022.csv  (236 entries)
 - fake-news-domains-Politifacts-fake-category-only-Jul2022.csv  (192 entries)
 - fake-news-domains-Wikipedia-fake-category-only-Jul2022.csv  (88 entries)
 - fake-news-domains-BuzzFeed-fake-category-only-Jul2022.csv  (257 entries)
 - fake-news-domains-all-Jul2022.csv  (1053 entries)
 - fake-news-domains-all-fake-category-only-Jul2022.csv  (582 entries)
 - fake-news-domains-intersection-Jul2022.csv  (15 entries)
 - fake-news-domains-intersection-fake-category-only-Jul2022.csv  (2 entries)
